In [29]:
import re
import numpy as np

text = open('./data/LSTM_HABR.txt', 'r', encoding='utf-8').read().lower()
text = re.sub(r'[^а-я\s\d.!\?-]+', '', text)
text = re.sub(r'\s+', ' ', text) 
text = re.sub(r'[!\?]+', '.', text)
sentences = text.split('.')

In [30]:
def fill_strings(x, y, window):
    for sentence in sentences:
        sentence += '.'
        for i in range(0, len(sentence) - window):
            x.append(sentence[i:i + window])
            y.append(sentence[i + window])
    return len(x)        

In [31]:
chars = set(text)
size_vocab = len(chars)
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

window = 25

X = []
Y = []
    
size_sentences = fill_strings(X, Y, window)
x_train = np.zeros((size_sentences, window, size_vocab), dtype=np.bool_)
y_train = np.zeros((size_sentences, size_vocab), dtype=np.bool_)

for i in range(0, size_sentences):
    x_cur = X[i]
    y_cur = Y[i]
    for j, ch in enumerate(x_cur):
        x_train[i, j, char_to_int[ch]] = True
    y_train[i, char_to_int[y_cur]] = True 
    
print(x_train.shape[1:])    

(25, 42)


# RNN

In [24]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
np.random.seed(42)
 
model = Sequential(
    [
        LSTM(100, input_shape=x_train.shape[1:], dropout=0.2, recurrent_dropout=0.2, return_sequences=True),
        LSTM(100),
        Dense(y_train.shape[1], activation='softmax')
    ]
)
model.compile(loss="categorical_crossentropy", optimizer='adam')
model.fit(x_train, y_train, batch_size=20, epochs=40)

Epoch 1/40
444/444 [==============================] - 26s 54ms/step - loss: 3.1900
Epoch 2/40
444/444 [==============================] - 24s 54ms/step - loss: 2.9525
Epoch 3/40
444/444 [==============================] - 27s 62ms/step - loss: 2.6953
Epoch 4/40
444/444 [==============================] - 30s 68ms/step - loss: 2.5581
Epoch 5/40
444/444 [==============================] - 35s 78ms/step - loss: 2.4442
Epoch 6/40
444/444 [==============================] - 31s 70ms/step - loss: 2.3582
Epoch 7/40
444/444 [==============================] - 29s 65ms/step - loss: 2.2765
Epoch 8/40
444/444 [==============================] - 32s 72ms/step - loss: 2.2008
Epoch 9/40
444/444 [==============================] - 29s 65ms/step - loss: 2.1295
Epoch 10/40
444/444 [==============================] - 28s 64ms/step - loss: 2.0666
Epoch 11/40
444/444 [==============================] - 37s 84ms/step - loss: 2.0110
Epoch 12/40
444/444 [==============================] - 28s 62ms/step - loss: 1.9666
E

In [32]:
index = np.random.randint(0, len(x_train) - 1)
sentence = X[index]
print('Start sentence: "' + sentence + '"')
for i in range(100):
    x_pred = np.zeros((1, window, size_vocab))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_to_int[char]] = 1.0
    preds = model.predict(x_pred)                    
    next_char = int_to_char[np.argmax(preds)]
    sentence = sentence[1:] + next_char
    print(next_char, end='')

Start sentence: "овые значения-кандидаты у"
станующих слово состояние ячейки объединяет в работе 2015 года провлемы долговременных зависимостей 

# Markov chane

In [33]:
from collections import defaultdict

window = 6
markov_X = []
markov_Y = []
fill_strings(markov_X, markov_Y, window)

nodes = defaultdict(lambda: defaultdict(lambda: 0))
for sentence, symbol in zip(markov_X, markov_Y):
    nodes[sentence][symbol] += 1

index = np.random.randint(0, len(markov_X) - 1)

sentence = markov_X[index]
print('Start sentence: "' + sentence + '"')
for i in range(100):
    next_chars_pool = [symbol for symbol in nodes[sentence]]
    probas = np.array([w for w in nodes[sentence].values()])
    probas = probas / probas.sum()
    if len(probas) == 0:
        break
    next_char = next_chars_pool[np.argmax(probas)]
    sentence = sentence[1:] + next_char
    print(next_char, end='')

Start sentence: "ный сл"
ой нейронные сети содержат обратные связи позволяющие сохранить а 0 полностью они были представлены 